# INITIAL APPROACH (WITH MINIMAL MEDIA)

Partimos de una lista de organismos aptos para el consumo humano, de los que deseamos comprobar si son capaces de crecer en un medio procedente de la hidrólisis del alga Ulva. Trabajaremos con aquellas especies que presentan modelos en AGORA2 o, en su defecto, que pertencen a un género con modelos en esta base de datos.

In [1]:
import openpyxl
import requests
import cobra
import os
from bs4 import BeautifulSoup
from cobra.io import read_sbml_model
from pathlib import Path
from cobra.medium import minimal_medium
import warnings
import json

### Lectura del Excel

Partimos de un archivo excel con una hoja "Especies_AGORA2" con 3 columnas:
- Especie: organismo GRAS
- Modelo en AGORA2: indica si existen modelos de cepas de dicha especie en dicho repositorio (Y) o no (N)
- Archivo: indica el nombre de los archivos de dichos modelos, separados por comas

Existe otra hoja "Géneros AGORA_2" con los géneros a los que pertenecen los anteriores organismos GRASS, indicando lo mismo pero a nivel de género (los modelos que existen en AGORA2 para dicho género). Esto es de utilidad para los organismos para los que no exista modelo como tal.

Leemos ese excel y creamos 3 listas:
- Organismos con modelo en AGORA2
- Organismos sin modelo en AGORA2 pero que pertenecen a un género del que existen organismos con modelo en AGORA2
- Organismos no soportados de ninguna manera en AGORA2, de los que no se pueden extraer conclusiones

In [2]:
#Leemos las hojas del excel y creamos las listas
def leer_excel(file_path):
    try:
        # Cargar el libro de trabajo de Excel
        workbook = openpyxl.load_workbook(file_path)
        
        # Seleccionar las hojas a trabajar
        especies_sheet = workbook["Especies_AGORA2"]
        generos_sheet = workbook["Géneros_AGORA2"]
        
        #Creamos los diccionarios
        especies_modelos = {}
        generos_modelos = {}
        no_modelos = []
        
        # Si el organismo (columna 1) tiene modelos (Y) creamos diccionario especies_modelos
        # Si no tiene modelos se busca el género
        # Si el género tiene modelos se añade al diccionario generos_modelos
        #Si no tiene nada se añade a la lista no_modelos
        for i, celda in enumerate(especies_sheet["B"][1:]):
            i = i+1 #para que corresponda con el numero de la fila
            nombre_especie = especies_sheet["A"][i].value
            if celda.value == "Y":
                modelos = especies_sheet["C"][i].value.split(",")
                especies_modelos[nombre_especie] = modelos
            else:
                #Nos quedamos con el genero y comprobamos si tiene modelos
                genero = nombre_especie.split("_")[0]
                #print(genero)
                soportado = False
                for j, gen in enumerate(generos_sheet["A"][1:]):
                    #print(gen.value)
                    j = j+1
                    if gen.value == genero and generos_sheet["B"][j].value == "Y":
                        soportado = True
                        modelos = generos_sheet["C"][j].value.split(",")
                        generos_modelos[nombre_especie] = modelos
                # Si el género no está soportado, se añade la especie a la lista de no_modelos
                if soportado == False:
                    no_modelos.append(nombre_especie)

        return especies_modelos, generos_modelos, no_modelos
    
    except Exception as e:
        print(f"Error al leer el archivo Excel: {e}")
        return None

### Descarga de modelos de AGORA2

In [3]:
def descargar_agora2(nombre_modelo):
    nombre_modelo = nombre_modelo
    url_padre = "https://www.vmh.life/files/reconstructions/AGORA2/version2.01/sbml_files/individual_reconstructions/"
    url = url_padre + nombre_modelo
    #Directorio donde se guardan los modelos
    nombre_directorio = "modelos_agora"
    directorio_actual = os.getcwd()
    ruta_directorio = os.path.join(directorio_actual, nombre_directorio)
    #Lo creamos si no existe
    if not os.path.exists(ruta_directorio):
        os.makedirs(ruta_directorio)
        models_dir = ruta_directorio
    else:
        models_dir = ruta_directorio
    #Si ya existe el archivo no hace falta descargarlo, se importa directamente
    ruta_archivo = os.path.join(str(models_dir), nombre_modelo)
    if os.path.exists(ruta_archivo):
        file_path = Path (models_dir)/(nombre_modelo)
        file_path = file_path.resolve()
        model = read_sbml_model(str(file_path))
        return model 
    #Si no existe se descarga y se importa
    else:
        try:
            # Realizar la solicitud GET a la URL del archivo
            respuesta = requests.get(url)

            # Verificar si la solicitud fue exitosa (código de estado 200)
            if respuesta.status_code == 200:
                # Guardar el archivo localmente
                with open(ruta_archivo, 'wb') as archivo_local:
                    archivo_local.write(respuesta.content)
                #print(f"Archivo descargado exitosamente como '{nombre_modelo}'.")
                #Determinamos el path al archivo
                file_path = Path (models_dir)/(nombre_modelo)
                file_path = file_path.resolve()
                model = read_sbml_model(str(file_path))
                return model

            else:
                print(f"Error al descargar el archivo. Código de estado: {respuesta.status_code}")

        except Exception as e:
            print("Error_modelo")

### Modificación de la composición del medio

El medio en el que deben crecer los organismos debe presentar las siguientes fuentes de carbono:
- glucosa
- rhamnosa
- xylosa
- ácido glucurónico
- hydroxymethylfurfural (HMF)
- furfural (poco)

Sin embargo, hay que tener en cuenta las auxotrofías de los organismos, que mantendremos. Vamos a eliminar todos los metabolitos del medio que no sean estrictamente necesarios para el crecimiento (medio mínimo). Si una fuente de carbono diferente de los de la lista está en el medio mínimo, lo eliminaremos también.

In [4]:
#Lista con las fuentes de carbono permitidas
fuentes_C = ["glucose", "rhamnose", "xylose", "glucuronic", "furfural", "hydroxymethylfurfural"]

#Para cada una de las reacciones de exchange evaluamos sus metabolitos
#Si alguno contiene C, añadimos la reacción a una lista, excepto si es alguno de las fuentes permitidas
#Tampoco lo añadimos si pertenece al medio mínimo
#Para todas las reacciones de esa lista se setearán los bound a 0 en el medio
#Es decir, eliminamos los exchange al menos que sean del medio mínimo o sean de alguno de los azúcares permitidos

#La siguiente función implementa lo anterior
#Además, devuelve la lista de exchanges a eliminar y de exchanges de fuentes de carbono admitidos
def update_media(model, fuentes_C = fuentes_C):
    exchange_elim = []
    #Obtenemos la lista de objetos metabolito del modelo asociados a fuentes de carbono permitidas o metabolitos del medio mínimo
    admitted_mets = []
    for admitted in fuentes_C:
        #Nos quedamos con los objetos metabolitos asociados a esa fuente
        for met in model.metabolites:
            if admitted in met.name.lower():
                admitted_mets.append(met)
    #Añadimos los objetos metabolito del medio mínimo, a menos que sean azúcares diferentes de los permitidos
    minimal = minimal_medium(model).index #Lista de identificadores de exchanges del medio mínimo
    for iden in minimal:
        rx = model.reactions.get_by_id(iden)
        metab = list(rx.metabolites.keys())[0]
        if not ("ose" in metab.name.lower() and not any(azucar in metab.name.lower() for azucar in fuentes_C)):
            admitted_mets.append(metab)
    
    #Para cada reacción de exchange que implique carbono decidimos si la mantenemos o no
    for ex in model.medium.keys():
        rx = model.reactions.get_by_id(ex) 
        no_elim = False
        #Comprobamos si es una fuente de carbono permitida o del medio mínimo
        if admitted_mets:
            met = list(rx.metabolites.keys())[0]
            for admit in admitted_mets:
                if met.id == admit.id: #Para que las nomenclaturas coincidan
                    no_elim = True

        #Se elimina la reacción si el metabolito no está admitido
        if no_elim == False:
            exchange_elim.append(ex)

    #Seteamos los bounds a 0 de las reacciones seleccionadas para eliminar las fuentes de carbono
    media = model.medium
    for rx in exchange_elim:
        media[rx] = 0
    model.medium = media
    
    #Nos quedamos con los nombres de los metabolitos del medio actualizado
    medio_final = []
    for iden in model.medium.keys():
        rx = model.reactions.get_by_id(iden)
        met = list(rx.metabolites)[0]
        medio_final.append(met.name)

    return medio_final, exchange_elim, admitted_mets

### Análisis final

Para cada uno de los organismos soportados en AGORA2 hacemos el análisis. Nos quedamos con un diccionario con los nombres de las especies y una tupla que contiene:
- Valor máximo de crecimiento (si solo hay una cepa que crece, este será el valor máximo)
- Cepas que crecen, con su valor de crecimiento y composición del medio

Creamos una función que realiza el análisis a partir de una lista de especies:

In [5]:
def growth_analysis(lista, fuentes_C = fuentes_C):
    resultado = {}
    modelos_error = []
    for especie, modelos in lista.items():
        crec_max = 0
        cepas_crec = {}
        #Recorremos todos los modelos de la especie y obtenemos el crecimiento máximo y las cepas que crecen
        #Para cada cepa que crece guardamos también los componentes del medio
        for nombre_modelo in modelos:
            try:
                model = descargar_agora2(nombre_modelo)
                medio_final = update_media(model)
                crec = model.slim_optimize()
                if crec != 0:
                    print(str(int(crec)),";", end="") #Estado
                    cepas_crec[nombre_modelo] = [crec, medio_final[0]] #Guardamos la cepa, su valor de crecimiento y el medio que necesita
                    if crec > crec_max:
                        crec_max = crec
                else:
                    print(".", end="") #Estado
            except:
                modelos_error.append(nombre_modelo)
                
        #Si alguna de las cepas crece, guardamos el resultado
        if crec_max != 0:
            tupla = (crec_max, cepas_crec)
            resultado[especie] = tupla
        #Estado del análisis
        print("\n", especie, " analizada")

    return resultado

# Análisis de organismos con modelo en AGORA2

In [6]:
#Importamos del excel y creamos las 3 listas
excel_path = "QPS_Lista_Danesa_List_June_2023_mod.xlsx"
especies_modelos = leer_excel(excel_path)[0]
generos_modelos = leer_excel(excel_path)[1]
no_modelos = leer_excel(excel_path)[2]

Realizamos el análisis descrito para la lista de especies que tienen algún modelo en AGORA2:

In [7]:
warnings.filterwarnings('ignore')
resultado = growth_analysis(especies_modelos)
resultado

Set parameter Username
Academic license - for non-commercial use only - expires 2025-02-12
86 ;83 ;47 ;42 ;47 ;
 Hafnia_alvei  analizada
41 ;
 Bifidobacterium_pseudolongum  analizada
55 ;
 Bifidobacterium_thermophilum  analizada
26 ;23 ;27 ;24 ;24 ;23 ;24 ;34 ;24 ;25 ;24 ;24 ;30 ;
 Bifidobacterium_adolescentis  analizada
38 ;37 ;40 ;30 ;35 ;34 ;43 ;36 ;43 ;33 ;29 ;32 ;40 ;27 ;27 ;27 ;27 ;
 Bifidobacterium_animalis  analizada
30 ;27 ;29 ;30 ;31 ;26 ;29 ;29 ;
 Bifidobacterium_bifidum  analizada
27 ;27 ;27 ;36 ;30 ;36 ;35 ;40 ;31 ;27 ;27 ;27 ;28 ;27 ;28 ;28 ;28 ;27 ;27 ;27 ;27 ;28 ;25 ;
 Bifidobacterium_breve  analizada
34 ;40 ;53 ;41 ;41 ;41 ;41 ;41 ;41 ;41 ;34 ;28 ;33 ;34 ;24 ;39 ;35 ;34 ;34 ;
 Bifidobacterium_longum  analizada
.
 Brevibacterium_linens  analizada
62 ;
 Brevibacterium_casei  analizada
45 ;
 Corynebacterium_ammoniagenes  analizada
18 ;18 ;
 Corynebacterium_casei  analizada
27 ;.43 ;
 Lactobacillus_acidophilus  analizada
17 ;
 Lactobacillus_amylolyticus  analizada
41 ;40 ;

{'Hafnia_alvei': (86.23460964184649,
  {'Hafnia_alvei_ATCC_51873.xml': [86.23460964184649,
    ['meso-2,6-Diaminoheptanedioate',
     'Adenosine',
     'L-alanyl-L-histidine',
     'L-arginine',
     'calcium(2+)',
     'L-cysteinylglycine',
     'Chloride',
     'Co2+',
     'Cu2+',
     'Cytidine',
     'Fe3+',
     'D-glucose',
     'Glycyl-L-asparagine',
     'Glycerol 3-phosphate',
     'Glycyl-L-methionine',
     'potassium',
     'magnesium',
     'Mn2+',
     'Menaquinone 8',
     'NMN',
     'O2',
     'octadecanoate (n-C18:0)',
     'Ornithine',
     'L-Rhamnose',
     'Siroheme',
     'sulfate',
     'Spermidine',
     'Thiamin',
     'L-tryptophan',
     'thiosulfate(2-)',
     'tetradecanoate (n-C14:0)',
     'aldehydo-D-xylose',
     'Zinc']],
   'Hafnia_alvei_BIDMC_31.xml': [83.09600043600281,
    ['Adenosine',
     'L-alanyl-L-histidine',
     'L-arginine',
     'calcium(2+)',
     'L-cysteinylglycine',
     'Chloride',
     'Co2+',
     'Cu2+',
     'Cytidine',
     'F

In [8]:
#Exportamos el resultado a json
with open('Initial_approach_minimal_result.json', 'w') as archivo:
    json.dump(resultado, archivo)

## Análisis de los resultados

In [9]:
#Importamos los resultados
with open('Initial_approach_minimal_result.json', 'r') as archivo:
    resultados = json.load(archivo)

Ordenamos los modelos de las especies que crecen de mejor a peor en función de la cantidad de metabolitos que forman parte del medio (cuanto menor número, mejor). En caso de empate, el mejor modelo será aquel con mayor crecimiento. Guardamos para cada especie un diccionario con el nombre del mejor modelo, la composición del medio necesario, el número de metabolitos del medio y su crecimiento predicho.

In [10]:
mejor_modelo = {}
for especie, info in resultados.items():
    info_mejor = {} #Diccionario con información del mejor modelo
    mejor = None
    tamaño_medio = 1000
    dicc_modelos = info[1]
    for mod, crec_medio in dicc_modelos.items():
        medio = crec_medio[1]
        if len(medio) < tamaño_medio:
            mejor = mod
        elif len(medio) == tamaño_medio:
                crec_mejor = dicc_modelos[mejor][0]
                crec_candidato = crec_medio[0]
                if crec_candidato > crec_mejor:
                    mejor = mod
    #Guardamos para cada especie un diccionario con información sobre el mejor modelo
    info_mejor["Nombre"] = mejor
    info_mejor["Medio"] = dicc_modelos[mejor][1]
    info_mejor["Tamaño_medio"] = len(dicc_modelos[mejor][1])
    info_mejor["Crecimiento"] = dicc_modelos[mejor][0]
    mejor_modelo[especie] = info_mejor

In [11]:
mejor_modelo

{'Hafnia_alvei': {'Nombre': 'Hafnia_alvei_HUMV_5920.xml',
  'Medio': ['meso-2,6-Diaminoheptanedioate',
   'Adenosine',
   'L-alanyl-L-histidine',
   'L-alanyl-L-threonine',
   'L-arginine',
   'calcium(2+)',
   'L-cysteinylglycine',
   'Chloride',
   'Co2+',
   'cytosine',
   'Cu2+',
   'Fe2+',
   'Fe3+',
   'D-glucose',
   'Glycyl-L-asparagine',
   'Glycerol 3-phosphate',
   'Glycylleucine',
   'Glycylphenylalanine',
   'Glycyl-L-tyrosine',
   'potassium',
   'L-methionyl-L-alanine',
   'magnesium',
   'Mn2+',
   'Menaquinone 8',
   'NMN',
   'octadecanoate (n-C18:0)',
   'Protoheme',
   '(R)-Pantothenate',
   'L-Rhamnose',
   'Siroheme',
   'sulfate',
   'Spermidine',
   'Thiamin',
   'L-tryptophan',
   'tetradecanoate (n-C14:0)',
   'aldehydo-D-xylose',
   'Zinc'],
  'Tamaño_medio': 37,
  'Crecimiento': 47.442286930309436},
 'Bifidobacterium_pseudolongum': {'Nombre': 'Bifidobacterium_pseudolongum_subsp_Pseudolongum_DSM_20099.xml',
  'Medio': ['Adenine',
   'L-alanyl-L-glutamine',
  

A partir del resultado anterior, ordenamos las especies de mejor a peor, nuevamente en función del número de metabolitos en el medio y del crecimiento predicho (del mejor modelo).

In [12]:
# Definir la función de ordenamiento
def criterio_ordenamiento(especie, mi_diccionario = mejor_modelo):
    return (mi_diccionario[especie]['Tamaño_medio'], -mi_diccionario[especie]['Crecimiento'])

# Obtener las claves ordenadas
especies_ordenadas = sorted(mejor_modelo.keys(), key=criterio_ordenamiento)

#La lista de especies, ordenadas de mejor a peor, queda:
print("Claves ordenadas:", especies_ordenadas)

Claves ordenadas: ['Bacillus_amyloliquefaciens', 'Bacillus_subtilis', 'Corynebacterium_casei', 'Streptococcus_thermophilus', 'Microbacterium_gubbeenense', 'Bifidobacterium_longum', 'Bacillus_mojavensis', 'Bacillus_licheniformis', 'Bifidobacterium_thermophilum', 'Bacillus_smithii', 'Bacillus_atrophaeus', 'Bacillus_velezensis', 'Bifidobacterium_adolescentis', 'Corynebacterium_ammoniagenes', 'Geobacillus_stearothermophilus', 'Bifidobacterium_bifidum', 'Lactococcus_lactis', 'Lysinibacillus_fusiformis', 'Bifidobacterium_breve', 'Enterococcus_faecalis', 'Bacillus_vallismortis', 'Bifidobacterium_animalis', 'Propionibacterium_freudenreichii', 'Bifidobacterium_pseudolongum', 'Hafnia_alvei', 'Bacillus_pumilus', 'Brevibacterium_casei', 'Enterococcus_faecium', 'Lactobacillus_amylolyticus', 'Leuconostoc_mesenteroides', 'Leuconostoc_lactis', 'Pediococcus_pentosaceus', 'Lactobacillus_helveticus', 'Lactobacillus_delbrueckii', 'Lactobacillus_johnsonii', 'Lactobacillus_gasseri', 'Leuconostoc_citreum', '

Comprobamos que realmente están ordenadas en función del tamaño medio y el crecimiento

In [13]:
print("Tamaño_medio\tCrecimiento")
for especie in especies_ordenadas:
    print(mejor_modelo[especie]["Tamaño_medio"],"\t", mejor_modelo[especie]["Crecimiento"])

Tamaño_medio	Crecimiento
26 	 30.746387429716346
26 	 25.051218708494904
26 	 18.129969249692667
27 	 21.587090912941996
28 	 44.516900607755886
28 	 34.414649951689135
28 	 32.493986636677704
29 	 60.26379189598235
29 	 55.93248115621994
29 	 40.442733887559164
30 	 37.79845020998603
30 	 30.710293496400556
30 	 30.702585378651595
32 	 45.0641849947545
32 	 33.72374954935539
32 	 29.89725859404358
32 	 26.38243464586346
32 	 25.520363850241523
33 	 25.374629475378395
35 	 45.56057922537775
35 	 28.765618643102016
35 	 27.525228753425456
36 	 59.06420456485616
36 	 41.748870935399424
37 	 47.442286930309436
38 	 66.9391866143888
38 	 62.578491109534504
39 	 33.335022046713306
39 	 17.439839126924827
40 	 64.97855699496846
41 	 27.797497889025585
41 	 23.47122750296193
42 	 41.40874223307062
42 	 32.529056448652966
43 	 41.631236641235354
44 	 54.12800793219259
44 	 53.31312790582507
44 	 40.82352021806777
44 	 36.847362993882726
45 	 56.603177071078115
45 	 43.11980613296209
45 	 24.72

Ahora creamos un diccionario "mejor_especie", con la misma información que "mejor modelo" pero ordenado:

In [14]:
mejor_especie = {}
for especie in especies_ordenadas:
    mejor_especie[especie] = mejor_modelo[especie]

In [15]:
mejor_especie

{'Bacillus_amyloliquefaciens': {'Nombre': 'Bacillus_amyloliquefaciens_subsp_amyloliquefaciens_DC_12.xml',
  'Medio': ['1,2-Diacyl-sn-glycerol (dioctadecanoyl, n-C18:0)',
   '4-hydroxybenzoate',
   'Adenosine',
   'L-alanyl-L-histidine',
   'L-arginine',
   'calcium(2+)',
   'L-cysteinylglycine',
   'Chloride',
   'Co2+',
   'Cu2+',
   'Deoxyguanosine',
   'Fe3+',
   'D-glucose',
   'Glycerol 3-phosphate',
   'glycogen, structure 2 (glycogenin-1,6-{7[1,4-Glc], 4[1,4-Glc]})',
   'potassium',
   'L-methionyl-L-alanine',
   'magnesium',
   'Mn2+',
   'O2',
   'Protoheme',
   'Riboflavin',
   'sulfate',
   'Thiamin',
   'aldehydo-D-xylose',
   'Zinc'],
  'Tamaño_medio': 26,
  'Crecimiento': 30.746387429716346},
 'Bacillus_subtilis': {'Nombre': 'Bacillus_subtilis_subsp_subtilis_str_SMY.xml',
  'Medio': ['1,2-Diacyl-sn-glycerol (dioctadecanoyl, n-C18:0)',
   '4-hydroxybenzoate',
   'L-alanyl-L-threonine',
   'L-arginine',
   'calcium(2+)',
   'L-cysteinylglycine',
   'Chloride',
   'Co2+',
  

## Exportamos los resultados

In [16]:
#Exportamos el resultado a json
with open('Mejores_especies_minimal.json', 'w') as archivo:
    json.dump(mejor_especie, archivo)

# Creación del Excel Final

En este archivo vamos a sintetizar la información que contiene el diccionario "mejor_especie"

In [19]:
# Crear un nuevo libro de Excel y seleccionar la hoja de trabajo
workbook = openpyxl.load_workbook("Resultado_Initial_Approach_Minimal.xlsx")
hoja_trabajo = workbook.active

# Rellenamos por cada especie(fila):
# Columna 1 con los nombres de las especies
# Columna 2 mejor modelo
# Columna 3Tamaño del medio
# Columna 3 Crecimiento predicho
for i, nombre  in enumerate(mejor_especie.keys(), start=2):
    info_especie = mejor_especie[nombre] #Diccionario de la especie
    hoja_trabajo.cell(row=i, column=1, value=nombre)
    hoja_trabajo.cell(row=i, column=2, value=info_especie["Nombre"])
    medio = ", ".join(info_especie["Medio"])
    hoja_trabajo.cell(row=i, column=3, value=medio)
    hoja_trabajo.cell(row=i, column=4, value=info_especie["Tamaño_medio"])
    hoja_trabajo.cell(row=i, column=5, value=info_especie["Crecimiento"])

# Guardar el libro de Excel
workbook.save('Resultado_Initial_Approach_Minimal.xlsx')

# DESGLOSE INITIAL APPROACH MINIMAL MEDIA

El objetivo es reorganizar la tabla obtenida del análisis anterior, de manera que se incluya una columna para cada metabolito. De esta manera, para cada organismo se indicará si el medio contiene un metabolito dado en función del valor de su columna:
- 1: lo contiene
- 0: no lo contiene

## Importamos resultado

In [20]:
#Importamos los resultados
with open('Mejores_especies_minimal.json', 'r') as archivo:
    resultados = json.load(archivo)

In [31]:
resultados

{'Bacillus_amyloliquefaciens': {'Nombre': 'Bacillus_amyloliquefaciens_subsp_amyloliquefaciens_DC_12.xml',
  'Medio': ['1,2-Diacyl-sn-glycerol (dioctadecanoyl, n-C18:0)',
   '4-hydroxybenzoate',
   'Adenosine',
   'L-alanyl-L-histidine',
   'L-arginine',
   'calcium(2+)',
   'L-cysteinylglycine',
   'Chloride',
   'Co2+',
   'Cu2+',
   'Deoxyguanosine',
   'Fe3+',
   'D-glucose',
   'Glycerol 3-phosphate',
   'glycogen, structure 2 (glycogenin-1,6-{7[1,4-Glc], 4[1,4-Glc]})',
   'potassium',
   'L-methionyl-L-alanine',
   'magnesium',
   'Mn2+',
   'O2',
   'Protoheme',
   'Riboflavin',
   'sulfate',
   'Thiamin',
   'aldehydo-D-xylose',
   'Zinc'],
  'Tamaño_medio': 26,
  'Crecimiento': 30.746387429716346},
 'Bacillus_subtilis': {'Nombre': 'Bacillus_subtilis_subsp_subtilis_str_SMY.xml',
  'Medio': ['1,2-Diacyl-sn-glycerol (dioctadecanoyl, n-C18:0)',
   '4-hydroxybenzoate',
   'L-alanyl-L-threonine',
   'L-arginine',
   'calcium(2+)',
   'L-cysteinylglycine',
   'Chloride',
   'Co2+',
  

## Nombres de las columnas

Los nombres de las columnas serán todos lo metabolitos diferentes que encontremos en los diferentes medios de todos los organismos.

In [22]:
metab_unique = [] #Lista de todos los metabolitos diferentes de todos los medios

for especie, info_especie in resultados.items():
    modelo = info_especie["Nombre"]
    medio = info_especie["Medio"]
    tamaño = info_especie["Tamaño_medio"]
    crecimiento = info_especie["Crecimiento"]
    for met in medio:
        if met not in metab_unique:
            metab_unique.append(met)

In [23]:
metab_unique

['1,2-Diacyl-sn-glycerol (dioctadecanoyl, n-C18:0)',
 '4-hydroxybenzoate',
 'Adenosine',
 'L-alanyl-L-histidine',
 'L-arginine',
 'calcium(2+)',
 'L-cysteinylglycine',
 'Chloride',
 'Co2+',
 'Cu2+',
 'Deoxyguanosine',
 'Fe3+',
 'D-glucose',
 'Glycerol 3-phosphate',
 'glycogen, structure 2 (glycogenin-1,6-{7[1,4-Glc], 4[1,4-Glc]})',
 'potassium',
 'L-methionyl-L-alanine',
 'magnesium',
 'Mn2+',
 'O2',
 'Protoheme',
 'Riboflavin',
 'sulfate',
 'Thiamin',
 'aldehydo-D-xylose',
 'Zinc',
 'L-alanyl-L-threonine',
 'cytosine',
 'Fe2+',
 'Glycyl-L-asparagine',
 'L-lysine',
 'Nicotinate',
 'pectins',
 'meso-2,6-Diaminoheptanedioate',
 'L-alanyl-L-leucine',
 'D-Glucosamine',
 'Glycylleucine',
 'Glycyl-L-tyrosine',
 'hydrogenphosphate',
 'Spermidine',
 'L-alanyl-L-glutamine',
 'Gly-Cys',
 'Menaquinone 7',
 '(R)-Pantothenate',
 'Pyridoxal',
 'Xanthine',
 'Glycyl-L-glutamine',
 'Glycyl-L-methionine',
 'Guanine',
 'Hypoxanthine',
 'Nitrate',
 'octadecanoate (n-C18:0)',
 'Adenosylcobalamin',
 'Larch 

## Creación del Excel

In [28]:
# Crear un nuevo libro de Excel y seleccionar la hoja de trabajo
workbook = openpyxl.load_workbook("Desglose_Initial_Approach_Minimal.xlsx")
hoja = workbook.active

# Rellenamos por cada especie(fila):
# Columna 1 con el nombre del mejor modelo
# Columna 2 tamaño del medio
# Columna 3 crecimiento
# Columnas 4-n metabolitos del medio

#Añadimos los nombres de las n columnas de metabolitos
for i, met  in enumerate(metab_unique, start=4):
    hoja.cell(row=1, column=i, value=met)

#Rellenamos las columnas
for i, nombre  in enumerate(resultados.keys(), start=2):
    info_especie = resultados[nombre] #Diccionario de la especie
    hoja.cell(row=i, column=1, value=info_especie["Nombre"])
    hoja.cell(row=i, column=2, value=info_especie["Tamaño_medio"])
    hoja.cell(row=i, column=3, value=info_especie["Crecimiento"])
    #Recorremos las columnas de los metabolitos
    for j, met in enumerate(metab_unique, start=4):
        if met in info_especie["Medio"]:
            hoja.cell(row=i, column=j, value=1)
        else:
            hoja.cell(row=i, column=j, value=0)

# Guardar el libro de Excel
workbook.save('Desglose_Initial_Approach_Minimal.xlsx')

## Organismos que consumen el máximo de sustratos

In [32]:
fuentes_C = ["glucose", "rhamnose", "xylose","glucuronic"]
better = {} #Especies que consumen dos azucares o más de la lista

for especie, info_modelo in resultados.items():
    medio = info_modelo["Medio"]
    conjunto = " ".join(medio).lower()
    consum = []
    n = 0
    for azucar in fuentes_C:
        if azucar in conjunto:
            n += 1
            consum.append(azucar)
    #Si consume 2 azúcares o más, añadir al diccionario
    if n >= 2:
        better[especie] = [n, consum]

In [34]:
for especie, info in better.items():
    if info[0] > 2:
        print(especie, " consume:", info[1])

Bifidobacterium_longum  consume: ['glucose', 'rhamnose', 'xylose']
Bacillus_mojavensis  consume: ['glucose', 'rhamnose', 'xylose']
Bifidobacterium_adolescentis  consume: ['glucose', 'rhamnose', 'xylose']
Bifidobacterium_bifidum  consume: ['glucose', 'rhamnose', 'xylose']
Bacillus_vallismortis  consume: ['glucose', 'rhamnose', 'xylose']
Hafnia_alvei  consume: ['glucose', 'rhamnose', 'xylose']
